In [1]:
import os

os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"
# os.environ["HTTP_PROXY"] = "http://127.0.0.1:7890"
# os.environ["HTTPS_PROXY"] = "http://127.0.0.1:7890"

In [23]:
import mindspore
import numpy as np
from mindnlp.evaluate import load
from mindnlp.engine import TrainingArguments, Trainer
from mindnlp.dataset import load_dataset
from mindnlp.transformers import AutoModelForSequenceClassification, AutoTokenizer
from mindnlp.peft import get_peft_model, PeftModel

In [4]:
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-chinese")
dataset = load_dataset("lansinuote/ChnSentiCorp")

In [5]:
def tokenize(text, label):
    output = tokenizer(
        text,
        padding="max_length",
        truncation=True,
        max_length=128,
    )
    return (
        output["input_ids"],
        output["token_type_ids"],
        output["attention_mask"],
        label,
    )

In [6]:
def datapipe(dataset):
    for split in dataset.keys():
        dataset[split] = dataset[split].map(
            tokenize,
            input_columns=["text", "label"],
            output_columns=["input_ids", "token_type_ids", "attention_mask", "labels"],
        )
    return dataset

In [7]:
dataset = datapipe(dataset)

In [8]:
id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}

In [9]:
model = AutoModelForSequenceClassification.from_pretrained(
    "google-bert/bert-base-chinese",
    num_labels=2,
    id2label=id2label,
    label2id=label2id,
)

The following parameters in checkpoint files are not loaded:
['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
The following parameters in models are missing parameter:
['classifier.weight', 'classifier.bias']
The following parameters in models are missing parameter:
['classifier.weight', 'classifier.bias']


In [10]:
from mindnlp.peft import LoHaConfig, TaskType

peft_config = LoHaConfig(
    task_type=TaskType.SEQ_CLS,
    inference_mode=False,
    r=8,
    alpha=32,
    target_cells=["query"],
    modules_to_save=["classifier"],
)

In [11]:
model = get_peft_model(model, peft_config)

In [12]:
train_loader = dataset["train"]
test_loader = dataset["test"]
val_loader = dataset["validation"]

In [13]:
accuracy_metric = load("accuracy")

In [14]:
# training and evaluation
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    preds = np.argmax(preds, axis=1)
    accuracy = accuracy_metric.compute(predictions=preds, references=labels)
    return {"accuracy": accuracy}

In [16]:
training_args = TrainingArguments(
    "output",
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    learning_rate=1e-4,
    num_train_epochs=10,
    evaluation_strategy="epoch",
)
trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=train_loader,
    eval_dataset=test_loader,
    compute_metrics=compute_metrics,
)

In [17]:
trainer.train()

  0%|          | 0/1500 [00:00<?, ?it/s]

 10%|█         | 150/1500 [00:47<06:46,  3.32it/s]

{'eval_loss': 0.5453808307647705, 'eval_accuracy': {'accuracy': 0.7475}, 'eval_runtime': 1.5895, 'eval_samples_per_second': 11.953, 'eval_steps_per_second': 0.629, 'epoch': 1.0}


                                                  
 20%|██        | 300/1500 [01:35<06:00,  3.33it/s]

{'eval_loss': 0.40547648072242737, 'eval_accuracy': {'accuracy': 0.8375}, 'eval_runtime': 1.4016, 'eval_samples_per_second': 13.556, 'eval_steps_per_second': 0.713, 'epoch': 2.0}


...

100%|██████████| 1500/1500 [07:56<00:00,  3.34it/s]

{'loss': 0.2402, 'learning_rate': 9.999999747378752e-05, 'epoch': 10.0}


                                                   
100%|██████████| 1500/1500 [07:59<00:00,  3.13it/s]

{'eval_loss': 0.23658284544944763, 'eval_accuracy': {'accuracy': 0.9116666666666666}, 'eval_runtime': 1.8105, 'eval_samples_per_second': 10.495, 'eval_steps_per_second': 0.552, 'epoch': 10.0}
{'train_runtime': 479.0418, 'train_samples_per_second': 200.4, 'train_steps_per_second': 3.131, 'train_loss': 0.3263415781656901, 'epoch': 10.0}


TrainOutput(global_step=1500, training_loss=0.3263415781656901, metrics={'train_runtime': 479.0418, 'train_samples_per_second': 200.4, 'train_steps_per_second': 3.131, 'train_loss': 0.3263415781656901, 'epoch': 10.0})

In [18]:
trainer.evaluate()

100%|██████████| 19/19 [00:01<00:00, 11.42it/s]


{'eval_loss': 0.23658284544944763,
 'eval_accuracy': {'accuracy': 0.9116666666666666},
 'eval_runtime': 1.9645,
 'eval_samples_per_second': 9.672,
 'eval_steps_per_second': 0.509,
 'epoch': 10.0}

In [19]:
model.save_pretrained("google-bert/bert-base-chinese_LoHa_SQE_CLS")

In [20]:
ckpt = f"google-bert/bert-base-chinese_LoHa_SQE_CLS/adapter_model.ckpt"

In [21]:
!du -h $ckpt

1.2M	google-bert/bert-base-chinese_LoHa_SQE_CLS/adapter_model.ckpt


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [25]:
new_model = AutoModelForSequenceClassification.from_pretrained(
    "google-bert/bert-base-chinese"
)
new_model = PeftModel.from_pretrained(
    new_model, "google-bert/bert-base-chinese_LoHa_SQE_CLS"
)

The following parameters in checkpoint files are not loaded:
['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
The following parameters in models are missing parameter:
['classifier.weight', 'classifier.bias']
The following parameters in models are missing parameter:
['classifier.weight', 'classifier.bias']
[WARNING] ME(395178:138011495900992,MainProcess):2024-05-26-22:59:38.437.747 [mindspore/train/serialization.py:1378] For 'load_param_into_net', 201 parameters in the 'net' are not loaded, because they are not in the 'parameter_dict', please check whether the network structure is consistent when training and loading checkpoint.
[WARNING] ME(395178:138011495900992,MainProcess):2024-05-26-22:59:38.438.061 [mindspore/train/serialization.py:1383] base_model.model.bert.embeddings.word_

In [39]:
text1 = "这个服务态度真心很差劲"
text2 = "他们的服务态度真的好棒"
output1 = tokenizer(
    text1, return_tensors="ms", truncation=True, padding="max_length", max_length=128
)
output2 = tokenizer(
    text2, return_tensors="ms", truncation=True, padding="max_length", max_length=128
)

In [40]:
model(**output1).logits.argmax().item()

0

In [41]:
model(**output2).logits.argmax().item()

1